# Tratamiento de datos duplicados, ausentes o no válidos

## Acerca de los datos
En este notebook, utilizaremos datos meteorológicos diarios que fueron tomados de la [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2) y alterados para introducir muchos problemas comunes a los que nos enfrentamos cuando trabajamos con datos.

*Nota: El NCEI forma parte de la Administración Nacional Oceánica y Atmosférica (NOAA) y, como se puede ver en la URL de la API, este recurso se creó cuando el NCEI se llamaba NCDC. Si la URL de este recurso cambiara en el futuro, puede buscar "NCEI weather API" para encontrar la actualizada.*

## Antecedentes de los datos

Significado de los datos:
- `PRCP`: precipitación en milímetros
- `SNOW`: nevadas en milímetros
- `SNWD`: profundidad de la nieve en milímetros
- `TMAX`: temperatura máxima diaria en grados Celsius
- `TMIN`: temperatura mínima diaria en grados Celsius
- `TOBS`: temperatura en el momento de la observación en grados Celsius
- `WESF`: equivalente en agua de la nieve en milímetros

Algunos datos importantes para orientarnos:
- Según el Servicio Meteorológico Nacional, la temperatura más fría jamás registrada en Central Park fue de -26,1 °C (-15 °F) el 9 de febrero de 1934: [fuente](https://www.weather.gov/media/okx/Climate/CentralPark/extremes.pdf)
- La temperatura de la fotosfera del Sol es de aproximadamente 5.505°C: [fuente](https://en.wikipedia.org/wiki/Sun)

## Configuración
Necesitamos importar `pandas` y leer los datos sucios para empezar:

In [51]:
import pandas as pd

df = pd.read_csv('data/dirty_data.csv')

## Encontrar datos problemáticos
Un buen primer paso es mirar algunas filas:

In [52]:
df

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
0,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
3,2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
4,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
...,...,...,...,...,...,...,...,...,...,...
760,2018-12-31T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,3.3,-3.3,-2.8,NaN,False
761,2018-12-31T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,3.3,-3.3,-2.8,NaN,False
762,2018-12-31T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,3.3,-3.3,-2.8,NaN,False
763,2018-12-31T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN


El examen de las estadísticas resumidas puede revelar valores extraños o ausentes:

In [53]:
df.describe()

c:\Users\ortiz\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
c:\Users\ortiz\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:4655: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF
count,765.000000,577.000000,577.0,765.000000,765.000000,398.000000,11.000000
mean,5.360392,4.202773,NaN,2649.175294,-15.914379,8.632161,16.290909
std,10.002138,25.086077,NaN,2744.156281,24.242849,9.815054,9.489832
min,0.000000,0.000000,-inf,-11.700000,-40.000000,-16.100000,1.800000
25%,0.000000,0.000000,NaN,13.300000,-40.000000,0.150000,8.600000
50%,0.000000,0.000000,NaN,32.800000,-11.100000,8.300000,19.300000
75%,5.800000,0.000000,NaN,5505.000000,6.700000,18.300000,24.900000
max,61.700000,229.000000,inf,5505.000000,23.900000,26.100000,28.700000


El método `info()` puede señalar valores omitidos y tipos de datos erróneos:

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765 entries, 0 to 764
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               765 non-null    object 
 1   station            765 non-null    object 
 2   PRCP               765 non-null    float64
 3   SNOW               577 non-null    float64
 4   SNWD               577 non-null    float64
 5   TMAX               765 non-null    float64
 6   TMIN               765 non-null    float64
 7   TOBS               398 non-null    float64
 8   WESF               11 non-null     float64
 9   inclement_weather  408 non-null    object 
dtypes: float64(7), object(3)
memory usage: 59.9+ KB


Podemos utilizar el método `isna()`/`isnull()` de la serie para encontrar nulos:

In [55]:
contain_nulls = df[df.SNOW.isna() | df.SNWD.isna() | df.TOBS.isna()| df.WESF.isna() | df.inclement_weather.isna()]
contain_nulls.shape[0] #Quiere decir que en alguna de las filas hay un nulo en al menos una de las columnas

765

In [56]:
contain_nulls.head(10)

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
0,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
3,2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
4,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
5,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
6,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
7,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
8,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
9,2018-01-05T00:00:00,?,0.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN


Tenga en cuenta que no podemos comprobar si tenemos `NaN` así:

In [57]:
df[df.inclement_weather == 'NaN'].shape[0]

0

Esto se debe a que en realidad es `np.nan`. Sin embargo, observe que esto tampoco funciona:

In [58]:
import numpy as np
df[df.inclement_weather == np.nan].shape[0]

0

Tenemos que utilizar uno de los métodos comentados anteriormente para que esto funcione:

In [59]:
df[df.inclement_weather.isna()]

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
0,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
9,2018-01-05T00:00:00,?,0.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN
10,2018-01-05T00:00:00,?,0.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
757,2018-12-29T00:00:00,?,21.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN
758,2018-12-29T00:00:00,?,21.3,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN
759,2018-12-30T00:00:00,?,0.0,NaN,NaN,5505.0,-40.0,NaN,NaN,NaN
763,2018-12-31T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN


Podemos encontrar `-inf`/`inf` comparando con `-np.inf`/`np.inf`:

In [60]:
df[df.SNWD.isin([-np.inf, np.inf])].shape[0]

577

En lugar de hacer esto para cada columna, podemos escribir una función que utilice un [diccionario de comprensión](https://www.python.org/dev/peps/pep-0274/) para comprobar todas las columnas por nosotros:

In [61]:
def get_inf_count(df:pd.DataFrame) -> dict:
    """Encontrar el número de valores inf/inf por columna en el marco de datos"""
    return {
        col: df[df[col].isin([np.inf, -np.inf])].shape[0] for col in df.columns
    }

get_inf_count(df)

{'date': 0,
 'station': 0,
 'PRCP': 0,
 'SNOW': 0,
 'SNWD': 577,
 'TMAX': 0,
 'TMIN': 0,
 'TOBS': 0,
 'WESF': 0,
 'inclement_weather': 0}

Antes de decidir cómo tratar los valores infinitos de la profundidad de nieve, debemos examinar las estadísticas de resumen de las nevadas, que constituyen una parte importante en la determinación de la profundidad de nieve:

In [62]:
pd.DataFrame({
    'np.inf Snow Depth': df[df.SNWD == np.inf].SNOW.describe(),
    '-np.inf Snow Depth': df[df.SNWD == -np.inf].SNOW.describe()
}).T

,count,mean,std,min,25%,50%,75%,max
np.inf Snow Depth,24.0,101.041667,74.498018,13.0,25.0,120.5,152.0,229.0
-np.inf Snow Depth,553.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


Veamos ahora las columnas `date` y `station`. Antes vimos el valor `?` de estación, así que sabemos que es el otro valor único. Sin embargo, vemos que algunas fechas están presentes 8 veces en los datos y sólo tenemos 324 días, lo que significa que también nos faltan días:

In [63]:
df.describe(include='object')

,date,station,inclement_weather
count,765,765,408
unique,324,2,2
top,2018-07-05T00:00:00,GHCND:USC00280907,False
freq,8,398,384


Podemos utilizar el método `duplicated()` para encontrar filas duplicadas:

In [64]:
df[df.duplicated()].shape[0]

284

El valor por defecto de `keep` es `'first'`, lo que significa que nos mostrará la primera fila en la que se vieron los datos duplicados; sin embargo, podemos pasar `False` para verla:

In [65]:
df[df.duplicated(keep=False)].shape[0] #si ponemos first mantiene uno de los duplicados pero en el caso de ponerle False borra todo

482

También podemos especificar las columnas a utilizar:

In [66]:
df[df.duplicated(['date', 'station'])].shape[0]

284

Veamos algunos duplicados. Sólo en los pocos valores que vemos aquí, sabemos que los 4 primeros están realmente en los datos 6 veces porque por defecto no estamos viendo su primera aparición:

In [67]:
df[df.duplicated()].head()

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
1,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2,2018-01-01T00:00:00,?,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
5,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
6,2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
8,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True


## Cuestiones atenuantes

### Manejo de datos duplicados
Como sabemos que tenemos los datos meteorológicos de NY y nos hemos dado cuenta de que sólo tenemos dos entradas para `estación`, podemos decidir eliminar la columna `estación` porque sólo nos interesan los datos meteorológicos. Sin embargo, cuando tratamos con datos duplicados, tenemos que pensar en las ramificaciones de eliminarlos. Observe que sólo tenemos datos para la columna `WESF` cuando la estación es `?`:

In [68]:
df[df.WESF.notna()]

,date,station,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
7,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
8,2018-01-04T00:00:00,?,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
58,2018-01-30T00:00:00,?,1.5,13.0,inf,5505.0,-40.0,NaN,1.8,True
137,2018-03-08T00:00:00,?,28.4,NaN,NaN,5505.0,-40.0,NaN,28.7,NaN
146,2018-03-13T00:00:00,?,3.0,13.0,inf,5505.0,-40.0,NaN,3.0,True
159,2018-03-21T00:00:00,?,6.6,114.0,inf,5505.0,-40.0,NaN,8.6,True
162,2018-03-21T00:00:00,?,6.6,114.0,inf,5505.0,-40.0,NaN,8.6,True
186,2018-04-02T00:00:00,?,14.0,152.0,inf,5505.0,-40.0,NaN,15.2,True
678,2018-11-16T00:00:00,?,47.0,152.0,inf,5505.0,-40.0,NaN,24.9,True
679,2018-11-16T00:00:00,?,47.0,152.0,inf,5505.0,-40.0,NaN,24.9,True


Si determinamos que no afectará a nuestro análisis, podemos utilizar `drop_duplicates()` para eliminarlos:

In [69]:
# 1. hacer que la fecha sea una fecha-hora
df.date = pd.to_datetime(df.date)

# 2. guarde esta información para más tarde
station_qm_wesf = df[df.station == '?'].drop_duplicates('date').set_index('date').WESF

# 3. ordenar al final
df.sort_values('station', ascending=False, inplace=True)

# 4. elimine los duplicados basándose en la columna de fecha manteniendo la primera ocurrencia
# que será la estación válida si tiene datos
df_deduped = df.drop_duplicates('date')

# 5. eliminar la columna de la estación porque hemos terminado con ella
df_deduped = df_deduped.drop(columns='station').set_index('date').sort_index()

# 6. tomar el WESF de la estación válida y volver a la estación ? si es nulo
df_deduped = df_deduped.assign(
    WESF=lambda x: x['WESF'].combine_first(station_qm_wesf)
)

df_deduped.shape

(324, 8)

In [70]:
station_qm_wesf

date
2018-01-01     NaN
2018-01-04    19.3
2018-01-05     NaN
2018-01-07     NaN
2018-01-08     NaN
              ... 
2018-12-27     NaN
2018-12-28     NaN
2018-12-29     NaN
2018-12-30     NaN
2018-12-31     NaN
Name: WESF, Length: 232, dtype: float64

In [47]:
df_deduped.head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
2018-01-04,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,NaN,True


Aquí utilizamos el método `combine_first()` para unir los valores a la primera entrada no nula; esto significa que si tuviéramos datos de ambas estaciones, tomaríamos primero el valor proporcionado por la estación nombrada y si (y sólo si) esa estación fuera nula tomaríamos el valor de la estación nombrada `?`. En la tabla siguiente se muestran algunos ejemplos de esta situación:

| estación GHCND:USC00280907 | estación ? | resultado de `combine_first()` |
| :---: | :---: | :---: |
| 1 | 17 | 1 |
| 1 | `NaN` | 1 |
| `NaN` | 17 | 17 |
| `NaN` | `NaN` | `NaN` |

Fíjate en la 4ª fila&mdash;tenemos `WESF` en el lugar correcto gracias al índice:

In [42]:
df_deduped.head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,-40.0,NaN,NaN,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,NaN,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,NaN,False
2018-01-04,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,NaN,True


### Tratamiento de los nulos
Podríamos eliminar los nulos, sustituirlos por algún valor arbitrario o imputarlos utilizando los datos circundantes. Cada una de estas opciones puede tener ramificaciones, por lo que debemos elegir sabiamente.

Podemos utilizar `dropna()` para eliminar filas en las que alguna columna tenga un valor nulo. Las opciones por defecto apenas nos dejan datos:

In [77]:
df_deduped.shape

(324, 8)

In [78]:
df_deduped.dropna().shape

(4, 8)

Si pasamos `how='all'`, podemos elegir que sólo se eliminen las filas en las que todo es nulo, pero esto no elimina nada:

In [79]:
df_deduped.dropna(how='all').shape

(324, 8)

Podemos utilizar sólo un subconjunto de columnas para determinar qué eliminar con el argumento `subset`:

In [80]:
df_deduped.dropna(
    how='all', subset=['inclement_weather', 'SNOW', 'SNWD'] #BORRA LAS FILAS A DONDE ESAS COLUMNAS TIENEN NAN
).shape

(293, 8)

Esto también se puede realizar a lo largo de las columnas, y también podemos exigir un cierto número de valores nulos antes de eliminar los datos:

In [83]:
df_deduped.dropna(axis='columns', thresh=df_deduped.shape[0] * .75).columns

Index(['PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'TOBS', 'inclement_weather'], dtype='object')

Podemos elegir rellenar los valores nulos en su lugar con `fillna()`:

In [84]:
df_deduped.loc[:,'WESF'].fillna(0, inplace=True)
df_deduped.head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,5505.0,-40.0,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,5505.0,-40.0,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


En este punto hemos hecho todo lo posible sin distorsionar los datos. Sabemos que nos faltan fechas, pero si reindexamos, no sabemos cómo rellenar los datos `NaN`. Con los datos meteorológicos, no podemos suponer que porque nevó un día nevará el siguiente o que la temperatura será la misma. Por este motivo, tenga en cuenta que los siguientes ejemplos sólo tienen fines ilustrativos: que podamos hacer algo no significa que debamos hacerlo.

Dicho esto, vamos a tratar de resolver algunos de los problemas que quedan con los datos de temperatura. Sabemos que cuando `TMAX` es la temperatura del Sol, debe ser porque no había ningún valor medido, así que vamos a sustituirlo por `NaN`. También lo haremos para `TMIN` que actualmente utiliza -40°C como marcador de posición cuando sabemos que la temperatura más fría jamás registrada en NYC fue de -15°F (-26,1°C) el 9 de febrero de 1934:

In [85]:
df_deduped = df_deduped.assign(
    TMAX=lambda x: x['TMAX'].replace(5505, np.nan),
    TMIN=lambda x: x['TMIN'].replace(-40, np.nan),
)

In [86]:
df_deduped

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,NaN,NaN,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True
...,...,...,...,...,...,...,...,...
2018-12-27,0.0,0.0,-inf,5.6,-2.2,-1.1,0.0,False
2018-12-28,11.7,0.0,-inf,6.1,-1.7,5.0,0.0,False
2018-12-29,21.3,NaN,NaN,NaN,NaN,NaN,0.0,NaN


También supondremos que la temperatura no cambiará drásticamente día a día. Tenga en cuenta que esto es en realidad una gran suposición, pero nos permitirá entender cómo funciona `fillna()` cuando proporcionamos una estrategia a través del parámetro `method`. El método `fillna()` nos da 2 opciones para el parámetro `method`:
- `'ffill'` para rellenar hacia adelante
- `'bfill'` para rellenar hacia atrás

*Note that `'nearest'` is missing because we are not reindexing.*

Aquí usaremos `'ffill'` para mostrar cómo funciona:

In [88]:
df_deduped.assign(
    TMAX=lambda x: x['TMAX'].ffill(),
    TMIN=lambda x: x['TMIN'].ffill()
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,-4.4,-13.9,NaN,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


Podemos utilizar `np.nan_to_num()` para convertir `np.nan` en 0 y `-np.inf`/`np.inf` en números finitos grandes negativos o positivos:

In [89]:
df_deduped.assign(
    SNWD=lambda x: np.nan_to_num(x['SNWD'])
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-1.797693e+308,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-1.797693e+308,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-1.797693e+308,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,1.797693e+308,NaN,NaN,NaN,19.3,True
2018-01-05,14.2,127.0,1.797693e+308,-4.4,-13.9,-13.9,0.0,True


Dependiendo de los datos con los que estemos trabajando, podemos utilizar el método `clip()` como alternativa a `np.nan_to_num()`. El método `clip()` permite limitar los valores a un umbral mínimo y/o máximo específico. Como `SNWD` no puede ser negativo, utilicemos `clip()` para imponer un límite inferior de cero. Para mostrar cómo funciona el límite superior, utilicemos el valor de `SNOW`:

In [90]:
df_deduped.assign(
    SNWD=lambda x: x['SNWD'].clip(0, x['SNOW'])
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,0.0,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,0.0,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,229.0,NaN,NaN,NaN,19.3,True
2018-01-05,14.2,127.0,127.0,-4.4,-13.9,-13.9,0.0,True


Podemos combinar `fillna()` con otros tipos de cálculos. Aquí sustituimos los valores perdidos de `TMAX` por la mediana de todos los valores `TMAX`, `TMIN` por la mediana de todos los valores `TMIN` y `TOBS` por la media de los valores `TMAX` y `TMIN`. Como colocamos `TOBS` en último lugar, tenemos acceso a los valores imputados para `TMIN` y `TMAX` en el cálculo:

In [91]:
df_deduped.assign(
    TMAX=lambda x: x['TMAX'].fillna(x['TMAX'].median()),
    TMIN=lambda x: x['TMIN'].fillna(x['TMIN'].median()),
    # average of TMAX and TMIN
    TOBS=lambda x: x['TOBS'].fillna((x['TMAX'] + x['TMIN']) / 2)
).head()

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,14.4,5.6,10.0,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.1,-12.2,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.9,-13.3,0.0,False
2018-01-04,20.6,229.0,inf,14.4,5.6,10.0,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.9,-13.9,0.0,True


In [92]:
df_deduped.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 324 entries, 2018-01-01 to 2018-12-31
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   PRCP               324 non-null    float64
 1   SNOW               288 non-null    float64
 2   SNWD               288 non-null    float64
 3   TMAX               249 non-null    float64
 4   TMIN               249 non-null    float64
 5   TOBS               249 non-null    float64
 6   WESF               324 non-null    float64
 7   inclement_weather  251 non-null    object 
dtypes: float64(7), object(1)
memory usage: 30.9+ KB


También podemos utilizar `apply()` para ejecutar el mismo cálculo en todas las columnas. Por ejemplo, vamos a rellenar todos los valores que faltan con la mediana móvil de 7 días de sus valores, estableciendo el número de periodos necesarios para el cálculo en 0 para asegurarnos de que no introducimos más valores `NaN` adicionales. Los cálculos continuos se tratarán en el capítulo 4, así que esto es un avance:

In [93]:
df_deduped.apply(
    # los cálculos continuos se tratarán en el capítulo 4, esta es una mediana continua de 7 días
    # fijamos min_periods (# de periodos requeridos para el cálculo) a 0 para que siempre obtengamos un resultado
    lambda x: x.fillna(x.rolling(7, min_periods=0).median())
).head(10) #va a rellenar las filas tomando al media de los 7 filas que van par adelante

,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.30,-16.1,-12.20,0.0,False
2018-01-03,0.0,0.0,-inf,-4.40,-13.9,-13.30,0.0,False
2018-01-04,20.6,229.0,inf,-6.35,-15.0,-12.75,19.3,True
2018-01-05,14.2,127.0,inf,-4.40,-13.9,-13.90,0.0,True
2018-01-06,0.0,0.0,-inf,-10.00,-15.6,-15.00,0.0,False
2018-01-07,0.0,0.0,-inf,-11.70,-17.2,-16.10,0.0,False
2018-01-08,0.0,0.0,-inf,-7.80,-16.7,-8.30,0.0,False
2018-01-10,0.0,0.0,-inf,5.00,-7.8,-7.80,0.0,False


La última estrategia que podemos probar es la interpolación con el método `interpolate()`. Especificamos el parámetro `method` con la estrategia de interpolación a utilizar. Hay muchas opciones, pero nos quedaremos con la predeterminada `'linear'`, que tratará los valores como espaciados uniformemente y colocará los valores perdidos en medio de los existentes. Tenemos algunos datos que faltan, así que primero reindexaremos. Los valores de `TMAX`, `TMIN` y `TOBS` son la media de los valores del día anterior (8 de enero) y del día posterior (10 de enero):

In [94]:
df_deduped\
    .reindex(pd.date_range('2018-01-01', '2018-12-31', freq='D'))\
    .apply(lambda x: x.interpolate())\
    .head(10)

C:\Users\ortiz\AppData\Local\Temp\ipykernel_9988\1910715606.py:3: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  .apply(lambda x: x.interpolate())\


,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WESF,inclement_weather
2018-01-01,0.0,0.0,-inf,NaN,NaN,NaN,0.0,NaN
2018-01-02,0.0,0.0,-inf,-8.3,-16.10,-12.20,0.0,False
2018-01-03,0.0,0.0,-inf,-4.4,-13.90,-13.30,0.0,False
2018-01-04,20.6,229.0,inf,-4.4,-13.90,-13.60,19.3,True
2018-01-05,14.2,127.0,inf,-4.4,-13.90,-13.90,0.0,True
2018-01-06,0.0,0.0,-inf,-10.0,-15.60,-15.00,0.0,False
2018-01-07,0.0,0.0,-inf,-11.7,-17.20,-16.10,0.0,False
2018-01-08,0.0,0.0,-inf,-7.8,-16.70,-8.30,0.0,False
2018-01-09,0.0,0.0,-inf,-1.4,-12.25,-8.05,0.0,NaN
2018-01-10,0.0,0.0,-inf,5.0,-7.80,-7.80,0.0,False


<hr>

<div style="display: flex; justify-content: space-between; margin-bottom: 10px;">
    <div style="width: 33.33%; text-align: left;">
        <a href="./4-reshaping_data.ipynb">
            <button>&#8592; Notebook Anterior</button>
        </a>
    </div>
    <div style="width: 33.33%; text-align: center;">
        <a href="../solutions/ch_03/solutions.ipynb">
            <button>Soluciones</button>
        </a>
    </div>
    <div style="width: 33.33%; text-align: right;">
        <a href="../ch_04/1-consulta_y_merge.ipynb">
            <button>Capitulo 4 &#8594;</button>
        </a>
    </div>
</div>

<hr>
